In [1]:
# Need nltk installed
# No need to run this part

import nltk
from nltk.corpus import stopwords

In [8]:
import numpy as np
import os
import os.path
import string
import re
import operator
import csv
import logging
import time
import random
import pickle
import glob

In [9]:
# Need gensim package installed
# No need to run this

import gensim
import gensim.models as g

Using gpu device 0: GeForce GTX 960M (CNMeM is disabled, cuDNN not available)


In [10]:
# Need PyLucene installed
# No need to run this

import lucene

from java.io import File 
from org.apache.lucene.analysis.standard import StandardAnalyzer 
from org.apache.lucene.document import Document, Field 
from org.apache.lucene.index import IndexWriter, IndexWriterConfig 
from org.apache.lucene.store import SimpleFSDirectory 
from org.apache.lucene.util import Version
from org.apache.lucene.search import IndexSearcher 
from org.apache.lucene.index import IndexReader 
from org.apache.lucene.queryparser.classic import QueryParser 

from org.apache.lucene import document, store, util


In [11]:
# Need Keras installed
from keras.models import Sequential, Model, Graph
from keras.layers import Dense, Activation, Input, Embedding, LSTM, merge, Dropout
from keras.regularizers import l2, activity_l2
from keras.layers.core import Dense, Flatten, Merge
from keras.utils.visualize_util import plot

Using Theano backend.


In [12]:
from scipy import spatial

In [13]:
import multiprocessing

cores = multiprocessing.cpu_count()
assert g.doc2vec.FAST_VERSION > -1
print cores

8


# Doc2Vec

## Preprocessing - To get all documents in one file for input to Doc2Vec
file already generated(allText.txt) - no need to run this

In [59]:
#make sure there is no filename already, otherwise overwrites..
def rename(dir, pattern):
    num = 1
    for pathAndFilename in glob.iglob(os.path.join(dir, pattern)):
        title, ext = os.path.splitext(os.path.basename(pathAndFilename))
        title = str(num)
        num = num + 1
        os.rename(pathAndFilename, 
                  os.path.join(dir, title + ext))

In [60]:
rename(r'/home/tarun/PE/booksCorpus/', r'*.txt')

In [5]:
cachedStopWords = stopwords.words("english")
print len(cachedStopWords)
updateCachedStopWords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'yo', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'or', 'until',   'once', 'here', 'there', 'other', 's', 't', 'can', 'will', 'just', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ma']

153


In [6]:
#dont forget to remove the allText.txt file everytime you run this code.
dirName = "/home/tarun/PE/newCorpus60/"
uniqueWords = {}
allText = ""
l = os.listdir(dirName)
for i in range(len(l)):
    l[i] = int(l[i][:-4])

l = sorted(l)
for i in range(len(l)):
    l[i] = str(l[i]) + ".txt"

for fname in l:
    with open(os.path.join(dirName, fname), 'r') as myFile:
        fileText = myFile.read().replace('\n', ' ') #each file's text - a paragraph like structure for us
        for char in ['.', '"', ',', '(', ')', '!', '?', ';', ':']:
            fileText = fileText.replace(char, '')
        fileText = fileText.replace("displaystyle", '')
        fileText = re.sub("""displaystyle""", "", fileText, re.I|re.S)
        fileText = re.sub("""[^0-9A-Za-z]""", ' ', fileText)

        fileText = re.sub("""\s+""", " ", fileText)
        fileText = re.sub("""\t+""", " ", fileText)
        fileText = ' '.join([word for word in fileText.split() if word not in updateCachedStopWords])
        #fileText = ' '.join([word for word in fileText.split()])
        allText = allText + fileText
        allText = allText + "\n"
    with open(os.path.join(dirName, fname), 'w') as myFile:
        myFile.write(fileText)
with open(os.path.join("/home/tarun/PE/", "newCorpus60.txt"), 'w') as myFile:
    myFile.write(allText)
#WOSP means without stopwords

In [42]:
with open(os.path.join("/home/tarun/PE/", "allTextBooks.txt"), 'r') as myFile:
    allTokens = []
    fileText = myFile.read()
    paras = fileText.split("\n")
    for eachPara in paras:
        #print eachPara
        tokens1 = nltk.word_tokenize(eachPara)
        if(len(tokens1) > 0):
            allTokens.append(tokens1)
with open(os.path.join("/home/tarun/PE/", "allTextTokensBooks.txt"), 'wb') as myFile:
    pickle.dump(allTokens, myFile)

In [43]:
with open(os.path.join("/home/tarun/PE/", "allTextTokensBooks.txt"), 'rb') as myFile:
    allTokens = pickle.load(myFile)

## Check the number of cores (to be given as paramter in the Doc2Vec training)

In [10]:
def tokenizeSentence(fileText):
    #fileText = "maddy is mad. tarun is super mad!"
    fileText = fileText.replace('\n', ' ') #each file's text - a paragraph like structure for us
    for char in ['.', '"', ',', '(', ')', '!', '?', ';', ':']:
        fileText = fileText.replace(char, '')
    fileText = fileText.replace("displaystyle", '')
    fileText = re.sub("""displaystyle""", "", fileText, re.I|re.S)
    fileText = re.sub("""[^0-9A-Za-z]""", ' ', fileText)

    fileText = re.sub("""\s+""", " ", fileText)
    fileText = re.sub("""\t+""", " ", fileText)
    tokens1 = nltk.word_tokenize(fileText)
    return tokens1

## Training the Doc2Vec Model
Model already trained with the below mentioned parameters. Two available models - model2 and model3!
Using model2. 

No need to run this code

In [16]:
tic = time.time()
#doc2vec parameters
vector_size = 100
window_size = 6
min_count = 1
sampling_threshold = 1e-5
negative_size = 5
train_epoch = 100
dm = 1 #0 = dbow; 1 = dmpv
worker_count = cores #number of parallel processes - number of cores

#input corpus
train_corpus = "/home/tarun/PE/newCorpus60.txt"

#output model
saved_path = "/home/tarun/PE/doc2vec/model3_100_newCorpus60_1min_6window_100trainEpoch.bin"

#enable logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

#train doc2vec model
docs = g.doc2vec.TaggedLineDocument(train_corpus)

#model2 - without dbow_words
#model3 - with dbow_words
model = g.Doc2Vec(docs, size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=worker_count, hs=0, dm=dm, negative=negative_size, dbow_words=1, dm_concat=1) #model3
#model = g.Doc2Vec(docs, size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=worker_count, hs=0, dm=dm, negative=negative_size, dm_concat=1) #model2

#save model
model.save(saved_path)
toc = time.time()
print toc - tic


2016-11-28 22:47:31,534 : INFO : collecting all words and their counts
2016-11-28 22:47:31,535 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2016-11-28 22:47:31,756 : INFO : PROGRESS: at example #10000, processed 556169 words (2517764/s), 33599 word types, 10000 tags
2016-11-28 22:47:31,958 : INFO : PROGRESS: at example #20000, processed 1111024 words (2757427/s), 48486 word types, 20000 tags
2016-11-28 22:47:32,160 : INFO : PROGRESS: at example #30000, processed 1667660 words (2756484/s), 60196 word types, 30000 tags
2016-11-28 22:47:32,358 : INFO : PROGRESS: at example #40000, processed 2224022 words (2815394/s), 69479 word types, 40000 tags
2016-11-28 22:47:32,617 : INFO : PROGRESS: at example #50000, processed 2777446 words (2147585/s), 77991 word types, 50000 tags
2016-11-28 22:47:32,941 : INFO : PROGRESS: at example #60000, processed 3332188 words (1715698/s), 84377 word types, 60000 tags
2016-11-28 22:47:33,157 : INFO : PROGRESS: at example #700

142.16680789


# Question Answering Model

## Building the Retrival system using PyLucene
Files already generated

No need to run this

In [15]:
# PATHS 
luceneIndexPath = '/home/tarun/PE/lucene/luceneIndexDirectoryNewCorpus60/'
corpus = '/home/tarun/PE/newCorpus60/'
trainingFilePath = '/home/tarun/PE/Dataset/training_set.tsv'

#inference hyper-parameters
start_alpha=0.01
infer_epoch=1000
    
lucene.initVM()

# ANALYZER
analyzer = StandardAnalyzer(util.Version.LUCENE_CURRENT) 

# DIRECTORY
directory = SimpleFSDirectory(File(luceneIndexPath))


#dont forget to remove the luceneIndexDirectory file everytime you run this code.

# INDEX WRITER
writerConfig = IndexWriterConfig(util.Version.LUCENE_CURRENT, analyzer) 
writer = IndexWriter(directory, writerConfig)

l = os.listdir(corpus)
for i in range(len(l)):
    l[i] = int(l[i][:-4])

l = sorted(l)
for i in range(len(l)):
    l[i] = str(l[i]) + ".txt"

#print writer.numDocs()
# INDEXING ALL DOCUMENTS/ARTICLES IN THE CORPUS
for fileName in l:
    #print fileName
    document = Document()
    article = os.path.join(corpus, fileName)
    content = open(article, 'r').read()
    document.add(Field("text", content, Field.Store.YES, Field.Index.ANALYZED))
    writer.addDocument(document)
#print writer.numDocs()
writer.close()

# INDEX READER
reader = IndexReader.open(directory)
searcher = IndexSearcher(reader)

# QUERYING FOR A QUESTION
queryParser = QueryParser(util.Version.LUCENE_CURRENT, "text", analyzer)

## Training File
## Retrieving the top n (=3) documents relevant to the question and getting question, paragraph and option vectors to create the input vector
Training file with input vectors and output already generated. 

No need to run this code

In [ ]:
tic = time.time()
trainingFilePath = '/home/tarun/PE/Dataset/training_set.tsv'

with open(trainingFilePath) as trainData:
    reader = csv.reader(trainData, delimiter="\t")
    header = 0

    storeQidDocs = {}
    
    for row in reader:
        if (header == 0):
            header = 1
            continue
        else:
            question = row[1]
            docNums = []
            # Retrieving the top n (=3) documents relevant to the question
            query = queryParser.parse(queryParser.escape(question))
            numPages = 3
            hits = searcher.search(query, numPages)
            for hit in hits.scoreDocs:
                doc_id = hit.doc
                docNums.append(doc_id)
            storeQidDocs[question] = docNums
            
# Store data (serialize)
with open('qidPagesTrain.pickle', 'wb') as handle:
    pickle.dump(storeQidDocs, handle, protocol=pickle.HIGHEST_PROTOCOL)

            
toc = time.time()
print toc - tic
    

### No need to run this code

In [ ]:

tic = time.time()
with open('/home/tarun/PE/Dataset/final_test_set.pkl', 'rb') as f:
    test = pickle.load(f)

for row in test:
    question = row[1]
    docNums = []
    # Retrieving the top n (=3) documents relevant to the question
    query = queryParser.parse(queryParser.escape(question))
    numPages = 3
    hits = searcher.search(query, numPages)
    for hit in hits.scoreDocs:
        doc_id = hit.doc
        docNums.append(doc_id)
    storeQidDocs[question] = docNums
            
# Store data (serialize)
with open('qidPagesTest.pickle', 'wb') as handle:
    pickle.dump(storeQidDocs, handle, protocol=pickle.HIGHEST_PROTOCOL)

            
toc = time.time()
print toc - tic
    

In [29]:
with open(os.path.join("/home/tarun/PE/corpus", "2.txt"), 'r') as myFile:
    fileText = tokenizeSentence(myFile.read())
    
check = model.infer_vector(fileText, alpha=start_alpha, steps=infer_epoch)
check20 = model.docvecs[0]
check21 = model.docvecs[1]
check22 = model.docvecs[2]
check23 = model.docvecs[3]

In [34]:
with open(os.path.join("/home/tarun/PE/corpus", "4.txt"), 'r') as myFile:
    fileText = myFile.read()
    
check2 = model.infer_vector(fileText, alpha=start_alpha, steps=infer_epoch)
check23 = model.docvecs[3]

In [ ]:
#print check, "\n\n\n"
#print check1
result = 1 - spatial.distance.cosine(check2, check23)
print result
#print check2
#print check23

In [18]:
modelPath="/home/tarun/PE/doc2vec/model3_100_NewCorpusWOSP_1min_2hit_4window.bin"
model = g.Doc2Vec.load(modelPath)
#inference hyper-parameters
tic = time.time()
start_alpha=0.01
infer_epoch=1000
trainingFilePath = '/home/tarun/PE/Dataset/training_set.tsv'

answers = ['A','B','C','D']

with open(trainingFilePath) as f:
    reader = csv.reader(f,delimiter = ",")
    data = list(reader)
    numQuestions = len(data) - 1

with open(trainingFilePath) as trainData:
    reader = csv.reader(trainData, delimiter="\t")
    header = 0
    storeInputVecInFile = np.zeros([numQuestions,700])
    storeOutputVecInFile = np.zeros([numQuestions,4])
    inputNum = 0
    for row in reader:
        if (header == 0):
            header = 1
            continue
        else:
            question = row[1]
            # Retrieving the top n (=3) documents relevant to the question
            query = queryParser.parse(queryParser.escape(question))
            question = tokenizeSentence(question)
            questionVec = model.infer_vector(question, alpha=start_alpha, steps=infer_epoch)
            numPages = 2
            hits = searcher.search(query, numPages)
            docVec = np.zeros(200)
            start=0
            output = []
            for hit in hits.scoreDocs:
                doc_id = hit.doc
                docV = model.docvecs[doc_id]
                docVec[start:start+100] = docV
                start = start+100
                
            optionList = [row[3], row[4], row[5], row[6]]
            inputVec = np.concatenate([docVec, questionVec])
            output = []
            for option in optionList:
                optionVec = model.infer_vector(tokenizeSentence(option), alpha=start_alpha, steps=infer_epoch)
                if(answers[optionList.index(option)] == row[2]):
                    output.append(1)
                else:
                    output.append(0)
                    
                #inputVec = merge([docVec, questionVec, optionVec], mode='concat')
                inputVec = np.concatenate([inputVec, optionVec])
            storeInputVecInFile[inputNum] = inputVec
            storeOutputVecInFile[inputNum] = output
            inputNum = inputNum + 1
            if(inputNum % 500 == 0):
                print inputNum, (time.time() - tic)
# Save the input vectors and output into a file
np.save('inputTokensModel3_100_NewCorpusWOSP_1min_2hit_4window.bin', storeInputVecInFile)
np.save('outputTokensModel3_100_NewCorpusWOSP_1min_2hit_4window.bin', storeOutputVecInFile)
toc = time.time()
print toc - tic

2016-10-31 17:43:57,415 : INFO : loading Doc2Vec object from /home/tarun/PE/doc2vec/model3_100_NewCorpusWOSP_1min_2hit_4window.bin
2016-10-31 17:43:57,885 : INFO : loading docvecs recursively from /home/tarun/PE/doc2vec/model3_100_NewCorpusWOSP_1min_2hit_4window.bin.docvecs.* with mmap=None
2016-10-31 17:43:57,886 : INFO : loading syn1neg from /home/tarun/PE/doc2vec/model3_100_NewCorpusWOSP_1min_2hit_4window.bin.syn1neg.npy with mmap=None
2016-10-31 17:43:58,021 : INFO : loading syn0 from /home/tarun/PE/doc2vec/model3_100_NewCorpusWOSP_1min_2hit_4window.bin.syn0.npy with mmap=None
2016-10-31 17:43:58,036 : INFO : setting ignored attribute syn0norm to None
2016-10-31 17:43:58,037 : INFO : setting ignored attribute cum_table to None


500 36.4908339977
1000 73.8320231438
1500 111.183315039
2000 149.428488016
2500 184.372644186
184.381067991


In [10]:
### Trying to make copies of the options and seeing if questions can be answered looking at the paragraph word by word

In [25]:
# Retrieving the top n (=3) documents relevant to the question
question = "What is the cause of change in motion or change in the state of motion?"
query = queryParser.parse(queryParser.escape(question))
question = tokenizeSentence(question)
questionVec = model.infer_vector(question, alpha=start_alpha, steps=infer_epoch)
numPages = 1
hits = searcher.search(query, numPages)
#docVec = np.zeros(200)
start=0
docText = ""
output = []
for hit in hits.scoreDocs:
    doc_id = hit.doc
    print doc_id, hit.toString()
    docT = searcher.doc(hit.doc)
    docText = docT.get("text").encode("utf-8")

listOfWords = docText.split(" ")
numWords = len(listOfWords)
vector_size = 100
num = 0
docVec = np.zeros([(numWords),vector_size])
for eachWord in listOfWords:
    docV = model.docvecs[doc_id]
    docVec[num] = docV
    num = num + 1



7465 doc=7465 score=0.8155148 shardIndex=-1
Newton s Laws Introduction The Big Idea Applied forces may cause objects to accelerate All forces come in pairs because they arise in the interaction of two objects you can t push without being pushed back The more force applied the greater the acceleration that is produced Objects with high masses are difficult to accelerate without a large force In the absence of applied forces objects move in a straight line at a constant speed or remain at rest In formal language Newton s First Law Every body continues in its state of rest or of uniform motion in a right straight line unless it is compelled to change that state by forces impressed upon it Newton s Second Law The change of motion is proportional to the motive force impressed and is made in the direction of the right straight line in which that force is impressed Newton s Third Law To every action there is always opposed an equal reaction or the mutual actions of two bodies upon each other 

In [ ]:
# Trial
modelPath="/home/tarun/PE/doc2vec/model3_100_NewCorpus_1min_4window.bin"
model = g.Doc2Vec.load(modelPath)
#inference hyper-parameters
tic = time.time()
start_alpha=0.01
infer_epoch=1000
trainingFilePath = '/home/tarun/PE/Dataset/training_set.tsv'

answers = ['A','B','C','D']

with open(trainingFilePath) as f:
    reader = csv.reader(f,delimiter = ",")
    data = list(reader)
    numQuestions = len(data) - 1

with open(trainingFilePath) as trainData:
    reader = csv.reader(trainData, delimiter="\t")
    header = 0
    storeInputVecInFile = np.zeros([numQuestions,700]) #question, docs, options
    storeOutputVecInFile = np.zeros([numQuestions,4])
    inputNum = 0
    for row in reader:
        if (header == 0):
            header = 1
            continue
        else:
            question = row[1]
            query = queryParser.parse(queryParser.escape(question))
            question = tokenizeSentence(question)
            questionVec = model.infer_vector(question, alpha=start_alpha, steps=infer_epoch)
            numPages = 1
            hits = searcher.search(query, numPages)
            docText = ""
            output = []
            for hit in hits.scoreDocs:
                doc_id = hit.doc
                print doc_id, hit.toString()
                docT = searcher.doc(hit.doc)
                docText = docT.get("text").encode("utf-8")

            listOfWords = docText.split(" ")
            numWords = len(listOfWords)
            vector_size = 100
            num = 0
            docVec = np.zeros([(numWords),vector_size])
            for eachWord in listOfWords:
                docV = model.docvecs[doc_id]
                docVec[num] = docV
                num = num + 1

                
            optionList = [row[3], row[4], row[5], row[6]]
            inputVec = np.concatenate([docVec, questionVec])
            output = []
            for option in optionList:
                optionVec = model.infer_vector(tokenizeSentence(option), alpha=start_alpha, steps=infer_epoch)
                if(answers[optionList.index(option)] == row[2]):
                    output.append(1)
                else:
                    output.append(0)
                    
                #inputVec = merge([docVec, questionVec, optionVec], mode='concat')
                inputVec = np.concatenate([inputVec, optionVec])
            storeInputVecInFile[inputNum] = inputVec
            storeOutputVecInFile[inputNum] = output
            inputNum = inputNum + 1
            if(inputNum % 500 == 0):
                print inputNum, (time.time() - tic)
# Save the input vectors and output into a file
np.save('inputTokensModel3_100_NewCorpusWOSP_1min_2hit_4window.bin', storeInputVecInFile)
np.save('outputTokensModel3_100_NewCorpusWOSP_1min_2hit_4window.bin', storeOutputVecInFile)
toc = time.time()
print toc - tic

In [ ]:
modelPath="/home/tarun/PE/doc2vec/model3_100_NewCorpusWOSP_1min_2hit_4window.bin"
model = g.Doc2Vec.load(modelPath)
#inference hyper-parameters
tic = time.time()
start_alpha=0.01
infer_epoch=1000
trainingFilePath = '/home/tarun/PE/Dataset/training_set.tsv'

answers = ['A','B','C','D']

with open(trainingFilePath) as f:
    reader = csv.reader(f,delimiter = ",")
    data = list(reader)
    numQuestions = len(data) - 1

with open(trainingFilePath) as trainData:
    reader = csv.reader(trainData, delimiter="\t")
    header = 0
    storeInputVecInFile = np.zeros([numQuestions,700])
    storeOutputVecInFile = np.zeros([numQuestions,4])
    inputNum = 0
    for row in reader:
        if (header == 0):
            header = 1
            continue
        else:
            question = row[1]
            # Retrieving the top n (=3) documents relevant to the question
            query = queryParser.parse(queryParser.escape(question))
            question = tokenizeSentence(question)
            questionVec = model.infer_vector(question, alpha=start_alpha, steps=infer_epoch)
            numPages = 2
            hits = searcher.search(query, numPages)
            docVec = np.zeros(200)
            start=0
            output = []
            for hit in hits.scoreDocs:
                doc_id = hit.doc
                print doc_id
                docV = model.docvecs[doc_id]
                docVec[start:start+100] = docV
                start = start+100
                
            optionList = [row[3], row[4], row[5], row[6]]
            inputVec = np.concatenate([docVec, questionVec])
            output = []
            for option in optionList:
                optionVec = model.infer_vector(tokenizeSentence(option), alpha=start_alpha, steps=infer_epoch)
                if(answers[optionList.index(option)] == row[2]):
                    output.append(1)
                else:
                    output.append(0)
                    
                #inputVec = merge([docVec, questionVec, optionVec], mode='concat')
                inputVec = np.concatenate([inputVec, optionVec])
            storeInputVecInFile[inputNum] = inputVec
            storeOutputVecInFile[inputNum] = output
            inputNum = inputNum + 1
            if(inputNum % 500 == 0):
                print inputNum, (time.time() - tic)
# Save the input vectors and output into a file
np.save('inputTokensModel3_100_NewCorpusWOSP_1min_2hit_4window.bin', storeInputVecInFile)
np.save('outputTokensModel3_100_NewCorpusWOSP_1min_2hit_4window.bin', storeOutputVecInFile)
toc = time.time()
print toc - tic

In [12]:
a = np.zeros(10)
b = np.ones(5)
c = np.ones(5)
a[0:5] = b
a[5:10] = c
a

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [58]:
modelPath="/home/tarun/PE/doc2vec/cbse.bin"
model = g.Doc2Vec.load(modelPath)
#inference hyper-parameters
tic = time.time()
start_alpha=0.01
infer_epoch=1000
trainingFilePath = '/home/tarun/PE/Dataset/passageCBSETrain.csv'

answers = ['A','B','C','D']

with open(trainingFilePath) as f:
    reader = csv.reader(f,delimiter = ",")
    data = list(reader)
    numQuestions = len(data) - 1
    print numQuestions

with open(trainingFilePath) as trainData:
    reader = csv.reader(trainData, delimiter=",")
    header = 0
    storeInputVecInFile = np.zeros([(numQuestions*4),900])
    storeOutputVecInFile = np.zeros([(numQuestions*4),1])
    inputNum = 0
    for row in reader:
        if (header == 0):
            header = 1
            continue
        else:
            #print row
            question = row[1]
            question = tokenizeSentence(question)
            questionVec = model.infer_vector(question, alpha=start_alpha, steps=infer_epoch)
            #docVec = np.zeros(300)
            passage = tokenizeSentence(row[7])
            docVec = model.infer_vector(passage, alpha=start_alpha, steps=infer_epoch)
            #print docVec.size
            optionList = [row[3], row[4], row[5], row[6]]
            for option in optionList:
                optionVec = model.infer_vector(tokenizeSentence(option), alpha=start_alpha, steps=infer_epoch)
                if(answers[optionList.index(option)] == row[2]):
                    output = 1
                else:
                    output = 0
                    
                #inputVec = merge([docVec, questionVec, optionVec], mode='concat')
                inputVec = np.concatenate([docVec, questionVec, optionVec])
                storeInputVecInFile[inputNum] = inputVec
                storeOutputVecInFile[inputNum] = output
                inputNum = inputNum + 1
                if(inputNum % 500 == 0):
                    print inputNum, (time.time() - tic)
# Save the input vectors and output into a file
np.save('inputTokensPassage1.npy', storeInputVecInFile)
np.save('outputTokensPassage1.npy', storeOutputVecInFile)
print len(storeInputVecInFile)
print len(storeOutputVecInFile)
toc = time.time()
print toc - tic

2016-10-27 18:58:19,015 : INFO : loading Doc2Vec object from /home/tarun/PE/doc2vec/cbse.bin
2016-10-27 18:58:19,378 : INFO : loading docvecs recursively from /home/tarun/PE/doc2vec/cbse.bin.docvecs.* with mmap=None
2016-10-27 18:58:19,379 : INFO : loading syn1neg from /home/tarun/PE/doc2vec/cbse.bin.syn1neg.npy with mmap=None
2016-10-27 18:58:19,457 : INFO : setting ignored attribute syn0norm to None
2016-10-27 18:58:19,457 : INFO : setting ignored attribute cum_table to None


34
136
136
0.634927988052


## Test File
## Retrieving the top n (=3) documents relevant to the question and getting question, paragraph and option vectors to create the input vector
Test file with input vectors generated.


No need to run this code

In [19]:
# TESTING

modelPath="/home/tarun/PE/doc2vec/model3_100_NewCorpusWOSP_1min_2hit_4window.bin"
model = g.Doc2Vec.load(modelPath)
#inference hyper-parameters
tic = time.time()
start_alpha=0.01
infer_epoch=1000


tic = time.time()
import pickle
with open('/home/tarun/PE/Dataset/final_test_set.pkl', 'rb') as f:
    test = pickle.load(f)

numQuestions = 0
for row in test:
    numQuestions = numQuestions + 1

storeInputVecInFileTest = np.zeros([numQuestions,700])
    
answers = ['A', 'B', 'C', 'D']
checkRows = []
inputNum = 0
for row in test:
    question = row[1]
    query = queryParser.parse(queryParser.escape(question))
    questionVec = model.infer_vector(tokenizeSentence(question), alpha=start_alpha, steps=infer_epoch)
    numPages = 1
    hits = searcher.search(query, numPages)
    docVec = np.zeros(200)
    start=0
    output = []
    for hit in hits.scoreDocs:
        doc_id = hit.doc
        docV = model.docvecs[doc_id]
        docVec[start:start+100] = docV
        start = start+100
    
    inputVec = np.concatenate([docVec, questionVec])
    for option in [row[2], row[3], row[4], row[5]]:
        optionVec = model.infer_vector(tokenizeSentence(option), alpha=start_alpha, steps=infer_epoch)
        inputVec = np.concatenate([inputVec, optionVec])
    storeInputVecInFileTest[inputNum] = inputVec
    inputNum = inputNum + 1
    if(inputNum % 1000 == 0):
        print inputNum, (time.time() - tic)
print (time.time() - tic)
np.save('inputTokensTestModel3_100_NewCorpusWOSP_1min_2hit_4window.bin', storeInputVecInFileTest)

2016-10-31 17:48:09,746 : INFO : loading Doc2Vec object from /home/tarun/PE/doc2vec/model3_100_NewCorpusWOSP_1min_2hit_4window.bin
2016-10-31 17:48:10,394 : INFO : loading docvecs recursively from /home/tarun/PE/doc2vec/model3_100_NewCorpusWOSP_1min_2hit_4window.bin.docvecs.* with mmap=None
2016-10-31 17:48:10,394 : INFO : loading syn1neg from /home/tarun/PE/doc2vec/model3_100_NewCorpusWOSP_1min_2hit_4window.bin.syn1neg.npy with mmap=None
2016-10-31 17:48:10,522 : INFO : loading syn0 from /home/tarun/PE/doc2vec/model3_100_NewCorpusWOSP_1min_2hit_4window.bin.syn0.npy with mmap=None
2016-10-31 17:48:10,537 : INFO : setting ignored attribute syn0norm to None
2016-10-31 17:48:10,538 : INFO : setting ignored attribute cum_table to None


1000 63.940874815
2000 124.569279909
3000 185.44337678
4000 245.862912893
5000 304.016122818
6000 363.020120859
7000 421.743694782
8000 482.004225969
9000 543.718870878
10000 603.339489937
11000 664.023602009
12000 724.164276838
13000 785.66147089
14000 843.732502937
15000 902.652908802
16000 965.539227962
17000 1026.9018929
18000 1087.19457889
19000 1146.87776494
20000 1206.42624378
21000 1267.29271579
1285.13073683


In [60]:
modelPath="/home/tarun/PE/doc2vec/cbse.bin"
model = g.Doc2Vec.load(modelPath)
#inference hyper-parameters
tic = time.time()
start_alpha=0.01
infer_epoch=1000

tic = time.time()

trainingFilePath = '/home/tarun/PE/Dataset/passageCBSETest.csv'

with open(trainingFilePath) as f:
    reader = csv.reader(f,delimiter = ",")
    data = list(reader)
    numQuestions = len(data) - 1
    print numQuestions
    
storeInputVecInFileTest = np.zeros([(numQuestions*4),900])
    
answers = ['A', 'B', 'C', 'D']
checkRows = []
inputNum = 0


with open(trainingFilePath) as trainData:
    test = csv.reader(trainData, delimiter=",")
    header = 0

    for row in test:
        
        if (header == 0):
            header = 1
            continue
        else:

            question = row[1]
            #print row
            questionVec = model.infer_vector(tokenizeSentence(question), alpha=start_alpha, steps=infer_epoch)
            passage = tokenizeSentence(row[7])
            docVec = model.infer_vector(passage, alpha=start_alpha, steps=infer_epoch)
            answerScores = []
            for option in [row[3], row[4], row[5], row[6]]:
                optionVec = model.infer_vector(tokenizeSentence(option), alpha=start_alpha, steps=infer_epoch)
                inputVec = np.concatenate([docVec, questionVec, optionVec])
                #print inputVec.size
                storeInputVecInFileTest[inputNum] = inputVec
                inputNum = inputNum + 1
                if(inputNum % 1000 == 0):
                    print inputNum, (time.time() - tic)
    print (time.time() - tic)
    np.save('inputTokensTestPassage1.npy', storeInputVecInFileTest)

2016-10-27 19:02:22,493 : INFO : loading Doc2Vec object from /home/tarun/PE/doc2vec/cbse.bin
2016-10-27 19:02:22,570 : INFO : loading docvecs recursively from /home/tarun/PE/doc2vec/cbse.bin.docvecs.* with mmap=None
2016-10-27 19:02:22,572 : INFO : loading syn1neg from /home/tarun/PE/doc2vec/cbse.bin.syn1neg.npy with mmap=None
2016-10-27 19:02:22,643 : INFO : setting ignored attribute syn0norm to None
2016-10-27 19:02:22,643 : INFO : setting ignored attribute cum_table to None


28
0.573496103287


## The code to be run

Loading the stored numpy arrays (input vectors and output)

In [9]:
modelPath="/home/tarun/PE/doc2vec/model3_100_NewCorpusWOSP_1min_2hit_4window.bin"
model = g.Doc2Vec.load(modelPath)

In [11]:
storeInputVecInFile = np.load('/home/tarun/PE/inputTokensModel3_100_NewCorpusWOSP_1min_2hit_4window.npy')
storeOutputVecInFile = np.load('outputTokensModel3_100_NewCorpusWOSP_1min_2hit_4window.npy')
storeInputVecInFileTest = np.load('inputTokensTestModel3_100_NewCorpusWOSP_1min_2hit_4window.npy.npy')

IOError: [Errno 2] No such file or directory: '/home/tarun/PE/inputTokensModel3_100_NewCorpusWOSP_1min_2hit_4window.npy'

## Building and Training the neural network for QA model

We now have the input and output vectors needed for training the QA model. The input to the neural network is a numpy array "storeInputVecInFile" which has input vectors (concatenation of paragraph, question and option vectors). The output for training is the numpy array "storeOutputVecInFile" which has 0/1 (0-wrong option) for each question-option pair. 

In [24]:
network = Sequential()

'''
network.add(Dense(output_dim=700, input_dim=900, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dense(500, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dense(250, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dense(50, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dense(1, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
'''

network.add(Dense(output_dim=500, input_dim=700, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dropout(0.3))
'''
network.add(Dense(500, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dense(500, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dropout(0.2))
network.add(Dense(500, init='uniform', activation='sigmoid'))
network.add(Dense(500, init='uniform', activation='sigmoid'))
network.add(Dropout(0.1))
network.add(Dense(500, init='uniform', activation='sigmoid', activity_regularizer=activity_l2(0.01)))
'''
network.add(Dense(4, init='uniform', activation='softmax', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))


'''
network.add(Dense(output_dim=900, input_dim=900, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dropout(0.3))
network.add(Dense(900, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dropout(0.3))
network.add(Dense(900, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dropout(0.3))
network.add(Dense(900, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dropout(0.3))
network.add(Dense(950, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dropout(0.3))
network.add(Dense(450, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dropout(0.3))
network.add(Dense(1, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
'''
network.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

#the lines above and below this comment should be run together, otherwise the parameters won't be reinitialised

network.fit(storeInputVecInFile, storeOutputVecInFile, nb_epoch = 1, batch_size = 1, verbose = 2)
#verbose = 2 is needed otherwise it gives error of I/O operations on a closed file


Epoch 1/1
8s - loss: 8.3831 - acc: 0.2540


## Predicting output for the test file

In [25]:
tic = time.time()
#storeInputVecInFileTest = np.load('inputTokensTestPassage1.npy')

ans = network.predict(storeInputVecInFileTest, batch_size=81000, verbose=0)
#changing the batch_size here doesnt make a difference "apparantly"... 
#better to have batch_size as 85191, to reduce the time
#ans1 = network.predict_proba(storeInputVecInFileTest, batch_size=1, verbose=0)

toc = time.time()
print (toc - tic)

0.477111101151


In [26]:
plot(network, to_file='network.png')

In [27]:
print ans[:20]

[[ 0.21925756  0.27385992  0.27601773  0.23086479]
 [ 0.21925755  0.27385995  0.27601773  0.23086479]
 [ 0.21925753  0.27385995  0.27601773  0.23086478]
 [ 0.21925761  0.27385992  0.2760177   0.23086481]
 [ 0.21925756  0.27385992  0.27601773  0.23086479]
 [ 0.21925752  0.27385998  0.27601776  0.23086479]
 [ 0.21925753  0.27385998  0.27601779  0.23086478]
 [ 0.21925755  0.27385995  0.27601773  0.23086479]
 [ 0.21925758  0.27385992  0.27601773  0.23086481]
 [ 0.21925755  0.27385995  0.27601773  0.23086479]
 [ 0.21925744  0.27386004  0.27601781  0.23086473]
 [ 0.21925753  0.27385995  0.27601773  0.23086479]
 [ 0.21925744  0.27385998  0.27601779  0.23086472]
 [ 0.21925759  0.27385992  0.2760177   0.23086481]
 [ 0.21925761  0.27385992  0.2760177   0.23086481]
 [ 0.21925755  0.27385995  0.27601773  0.23086479]
 [ 0.21925753  0.27385995  0.27601773  0.23086479]
 [ 0.21925752  0.27385998  0.27601776  0.23086476]
 [ 0.21925755  0.27385995  0.27601773  0.23086479]
 [ 0.21925753  0.27385995  0.27

### Trying the graphical Model

In [15]:

leftNetwork = Sequential()
leftNetwork.add(Dense(output_dim=700, input_dim=900, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
leftNetwork.add(Dense(450, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
leftNetwork.add(Dense(300, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))

rightNetwork = Sequential()
rightNetwork.add(Dense(output_dim=700, input_dim=900, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))

mergedNetwork = Sequential()
merged = Merge([leftNetwork, rightNetwork, network], mode='concat',concat_axis=1)
mergedNetwork.add(merged)
mergedNetwork.add(Activation('relu'))
mergedNetwork.add(Dense(250, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
mergedNetwork.add(Dense(50, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
mergedNetwork.add(Dense(1, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
mergedNetwork.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

#the lines above and below this comment should be run together, otherwise the parameters won't be reinitialised

mergedNetwork.fit([storeInputVecInFile, storeInputVecInFile, storeInputVecInFile], storeOutputVecInFile, nb_epoch = 1, batch_size = 2500, verbose = 2)
#verbose = 2 is needed otherwise it gives error of I/O operations on a closed file


Epoch 1/1
0s - loss: 8331.9783 - acc: 0.7500


In [19]:
network = Sequential()

network.add(Dense(output_dim=900, input_dim=900, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dropout(0.3))
network.add(Dense(900, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dropout(0.3))
network.add(Dense(900, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dropout(0.3))
network.add(Dense(900, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dropout(0.3))
network.add(Dense(950, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dropout(0.3))
network.add(Dense(450, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
network.add(Dropout(0.3))
network.add(Dense(1, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))

#network.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

leftNetwork = Sequential()
leftNetwork.add(Dense(output_dim=700, input_dim=900, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
leftNetwork.add(Dense(450, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
leftNetwork.add(Dense(300, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))

rightNetwork = Sequential()
rightNetwork.add(Dense(output_dim=700, input_dim=900, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))

mergedNetwork = Sequential()
merged = Merge([leftNetwork, rightNetwork, network], mode='concat',concat_axis=1)
mergedNetwork.add(merged)
mergedNetwork.add(Activation('relu'))
mergedNetwork.add(Dense(250, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
mergedNetwork.add(Dense(50, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
mergedNetwork.add(Dense(1, init='uniform', activation='sigmoid', bias=True, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
mergedNetwork.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

#the lines above and below this comment should be run together, otherwise the parameters won't be reinitialised

mergedNetwork.fit([storeInputVecInFile, storeInputVecInFile, storeInputVecInFile], storeOutputVecInFile, nb_epoch = 1, batch_size = 2500, verbose = 2)
#verbose = 2 is needed otherwise it gives error of I/O operations on a closed file
tic = time.time()
#storeInputVecInFileTest = np.load('inputTokensTest2.npy')

ans = mergedNetwork.predict([storeInputVecInFileTest, storeInputVecInFileTest, storeInputVecInFileTest], batch_size=85191, verbose=0)
#changing the batch_size here doesnt make a difference "apparantly"... 
#better to have batch_size as 85191, to reduce the time
#ans1 = network.predict_proba(storeInputVecInFileTest, batch_size=1, verbose=0)

toc = time.time()
print (toc - tic)

Epoch 1/1
0s - loss: 21620.5413 - acc: 0.6230
31.876032114


In [18]:
plot(mergedNetwork, to_file='network.png')

In [26]:
tic = time.time()
storeInputVecInFileTest = np.load('inputTokensTest2.npy')

ans = mergedNetwork.predict([storeInputVecInFileTest, storeInputVecInFileTest], batch_size=85191, verbose=0)
#changing the batch_size here doesnt make a difference "apparantly"... 
#better to have batch_size as 85191, to reduce the time
#ans1 = network.predict_proba(storeInputVecInFileTest, batch_size=1, verbose=0)

toc = time.time()
print (toc - tic)

4.43773603439


## Writing output to a csv file (in the format to upload on Kaggle)

In [76]:
import pickle
with open('/home/tarun/PE/Dataset/final_test_set.pkl', 'rb') as f:
    test = pickle.load(f)

numQuestions = 0
for row in test:
    numQuestions = numQuestions + 1

    
submissionFile = open("submit.csv", "w")
writer = csv.writer(submissionFile, delimiter=',')
writer.writerow(['id', 'correctAnswer'])
qIds = []
answers = ['A', 'B', 'C', 'D']
for row in test:
    questionId = row[0]
    qIds.append(questionId)
    
for q in range(numQuestions):
    answerScores = []
    opAProb = ans[q*4]
    opBProb = ans[q*4 + 1]
    opCProb = ans[q*4 + 2]
    opDProb = ans[q*4 + 3]
    answerScores.append(opAProb)
    answerScores.append(opBProb)
    answerScores.append(opCProb)
    answerScores.append(opDProb)
    writer.writerow([qIds[q] , answers[answerScores.index(np.max(answerScores))]])
    
submissionFile.close()

In [67]:
submissionFile = open("checkTest.csv", "w")
writer = csv.writer(submissionFile, delimiter=',')
writer.writerow(['id', 'correctAnswer'])
qIds = []
for i in range(0,28):
    qIds.append(i+1)
    
answers = ['A', 'B', 'C', 'D']
print numQuestions, qIds
for q in range(numQuestions):
    answerScores = []
    opAProb = ans[q*4]
    opBProb = ans[q*4 + 1]
    opCProb = ans[q*4 + 2]
    opDProb = ans[q*4 + 3]
    answerScores.append(opAProb)
    answerScores.append(opBProb)
    answerScores.append(opCProb)
    answerScores.append(opDProb)
    writer.writerow([qIds[q] , answers[answerScores.index(np.max(answerScores))]])
    
submissionFile.close()

28 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]


## Checking accuracy of doc2vec model - no need to run this part

In [ ]:
modelPath="/home/tarun/PE/doc2vec/model2.bin"
modelLoad = g.Doc2Vec.load(modelPath)

In [ ]:
print modelLoad.similarity("heart", "blood")
print "\n"
print modelLoad.most_similar("blood")
print "\n"
print modelLoad.docvecs.most_similar(2)
#del modelLoad

In [10]:
a = np.array([1,2,3])
b = np.array([4,5,6, 7])
l = [a,b]
c = np.array(l)
print c
print type(c)

[array([1, 2, 3]) array([4, 5, 6, 7])]
<type 'numpy.ndarray'>


In [12]:
np.save('c.npy', c)

In [13]:
check = np.load('c.npy')

In [14]:
print check, type(check)

[array([1, 2, 3]) array([4, 5, 6, 7])] <type 'numpy.ndarray'>


In [3]:
import numpy as np
a = np.array([[4,3,5,1], [2,6,5,1],[8,3,2,4]])